# Exercício de regressão linear



In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [22]:
# Lendo dados de treino
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

https://spark.apache.org/docs/2.2.1/api/java/org/apache/spark/ml/source/libsvm/LibSVMDataSource.html

Na verdade, eles não são muito populares ao trabalhar com conjuntos de dados em Python, mas a documentação do Spark os usa muito por causa de sua formatação. 

In [23]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Este é o formato que o Spark espera. Duas colunas com os nomes "label" e "features".

A coluna "label" precisa ter o rótulo numérico, um valor numérico de regressão ou um valor numérico que corresponda a um agrupamento de classificação. 

A coluna de features possui dentro dela um vetor de todos os recursos que pertencem a essa linha. Normalmente, o que acabamos fazendo é combinar as várias colunas de recursos que temos em uma única coluna de 'features' usando as transformações de dados que aprendemos.

Vamos continuar trabalhando neste exemplo simples!

In [25]:
# Estes são os valores padrão para featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# Verifique a documentação com Shift + Tab para mais informações!

In [7]:
# treinando o modelo
lrModel = lr.fit(training)

In [8]:
# Imprime os coeficientes e intercepta para regressão linear
print("Coeficientes: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coeficientes: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


Existe um atributo de resumo que contém ainda mais informações!

In [9]:
# Resuma o modelo sobre o conjunto de treinamento e imprima algumas métricas
trainingSummary = lrModel.summary

Muitas informações, aqui estão alguns exemplos:

In [10]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Divisões de treinamento / teste

Mas espere! Cometemos um grande erro, nunca separamos nosso conjunto de dados em um conjunto de treinamento e teste. Em vez disso, treinamos em TODOS os dados, algo que geralmente queremos evitar. Lembre-se de que não teremos uma avaliação justa de nosso modelo julgando como ele se sai novamente com os mesmos dados em que foi treinado!

Felizmente, o Spark DataFrames tem um método quase muito conveniente de dividir os dados! Vamos ver isso:

In [12]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [13]:
# Passe na divisão entre treinamento / teste como uma lista.
# Não correto, mas geralmente 70/30 ou 60/40 divisões são usadas.
# Dependendo de quantos dados você tem e quão desequilibrado eles estão.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [14]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [15]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
|-15.437384793431217|(10,[0,1,2,3,4,5,...|
|-13.420594775890757|(10,[0,1,2,3,4,5,...|
| -13.15333560636553|(10,[0,1,2,3,4,5,...|
|-13.039928064104615|(10,[0,1,2,3,4,5,...|
|-12.977848725392104|(10,[0,1,2,3,4,5,...|
|-12.491442077546413|(10,[0,1,2,3,4,5,...|
| -12.41094640284016|(10,[0,1,2,3,4,5,...|
+----------

In [15]:
unlabeled_data = test_data.select('features')

In [16]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Agora só treinamos no train_data

In [17]:
correct_model = lr.fit(train_data)

Agora podemos obter diretamente um objeto .summary usando o método evaluate:

In [18]:
test_results = correct_model.evaluate(test_data)

In [19]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-27.120702852243237|
|  -19.5292235629075|
|-19.068358505191988|
|-19.235368296661854|
|-20.591635894834795|
|-16.837352496695868|
| -16.57504797746008|
| -17.54417948967125|
|-16.111928371629325|
|-12.928939612929433|
| -13.34417233288924|
| -12.05824615703504|
|-14.520838749043465|
|-11.586843966213083|
|-12.050735005420018|
|-12.892983743212543|
|-13.800445821557389|
| -8.345838227126873|
|-12.118013508867028|
|-12.152205444030127|
+-------------------+
only showing top 20 rows

RMSE: 10.204018935035988


Bem, isso é bom, mas realisticamente vamos querer testar esse modelo com dados não rotulados, afinal, esse é o ponto principal da construção do modelo em primeiro lugar. Podemos fazer isso novamente com uma chamada de método conveniente, neste caso, transform (). Que estava realmente sendo chamado dentro do método evaluate (). Vamos ver em ação:

In [20]:
predictions = correct_model.transform(unlabeled_data)

In [21]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.9253151855333959|
|(10,[0,1,2,3,4,5,...|  -3.308236854011841|
|(10,[0,1,2,3,4,5,...| -0.8046325328764168|
|(10,[0,1,2,3,4,5,...| -0.4319503187098679|
|(10,[0,1,2,3,4,5,...|  1.7457134219362138|
|(10,[0,1,2,3,4,5,...| -0.9662736919686488|
|(10,[0,1,2,3,4,5,...|  -0.751672755215868|
|(10,[0,1,2,3,4,5,...|   2.209412009748908|
|(10,[0,1,2,3,4,5,...|  1.0554453970868922|
|(10,[0,1,2,3,4,5,...|-0.49165516296132333|
|(10,[0,1,2,3,4,5,...| 0.30424426878462557|
|(10,[0,1,2,3,4,5,...|  -0.919602568357063|
|(10,[0,1,2,3,4,5,...|   2.020064963688411|
|(10,[0,1,2,3,4,5,...| -0.5075073123221763|
|(10,[0,1,2,3,4,5,...|   0.145748102744903|
|(10,[0,1,2,3,4,5,...|  1.0148162432457026|
|(10,[0,1,2,3,4,5,...|  1.9733728251648186|
|(10,[0,1,2,3,4,5,...| -3.2947114507619535|
|(10,[0,1,2,3,4,5,...|   0.502238243851401|
|(10,[0,1,2,3,4,5,...|  1.568894